Run in order:
1. Prereqs
2. Loading Dataset and model
3. In "For any Theme" write your preferred theme in (theme ="")
4. Run rest of the cells in the "For any theme" block
5. Write your query in the Analysis sub-block

# Prereqs


In [ ]:
!pip install transformers faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 114.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.1 MB/s eta 0:00:00


In [ ]:
! pip install annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.0/648.0 KB 18.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.1-cp38-cp38-linux_x86_64.whl size=581429 sha256=c64364b42826893163e4fe0516dd5e2d636d328686f637bb2f43be7c9d51835f
  Stored in directory: /root/.cache/pip/wheels/f9/93/19/30511c4a9ae6b4937455a134c34a39e13943e2c6f46fcd2ed2
Successfully built annoy


In [ ]:
import csv
import requests
import pandas as pd

In [ ]:
import time

In [ ]:
from transformers import AutoModel, AutoTokenizer
from pprint import pprint

import faiss
import annoy
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
import torch

In [ ]:
import numpy as np 

# Loading dataset and model

In [ ]:
import csv
import requests

# load training dataset
def load_data():
    CSV_URL = 'https://drive.google.com/u/0/uc?id=1Z-yb752A3o7b9dqrGt24XU0sl53FVqya&export=download'

    with requests.Session() as s:
        download = s.get(CSV_URL)
        decoded_content = download.content.decode('utf-8')
        cr = csv.reader(decoded_content.splitlines(), delimiter=',')
        train_data = list(cr)

    print(f"Number of examples = {len(train_data)}")
    ans, noans = 0, 0
    for x in train_data:
        if x[4] == 'False':
            noans += 1
        else:
            ans += 1
    print(f"\tAnswerable questions = {ans}")
    print(f"\tNon-Answerable questions = {noans}\n")
    print("Examples:")
    for i in [0, 1000, 1300]:
        print(' | '.join(train_data[i][:2]), ' | ', train_data[i][2][:20] + '...', ' | ', ' | '.join(train_data[i][3:]))
    return train_data

In [ ]:
def load_theme_wise_data(train_data):
    theme_wise_data = {}
    for x in train_data[1:]:
        if x[1] not in theme_wise_data:
            theme_wise_data[x[1]] = {
                'para': [],
                'ques': [],
                'ans': []
            }
        if x[2] not in theme_wise_data[x[1]]['para']:
            theme_wise_data[x[1]]['para'].append(x[2])
        theme_wise_data[x[1]]['ques'].append(x[3])
        # ans contains a list -> [Para_Number, Answer_possible, Answer_text, Answer_start]
        theme_wise_data[x[1]]['ans'].append([theme_wise_data[x[1]]['para'].index(x[2])] + x[4:])
    print(f'\nTotal {len(theme_wise_data)} themes present.')
    return theme_wise_data

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-qa/3"
model = hub.load(module_url)

In [ ]:
metric = faiss.METRIC_INNER_PRODUCT 

In [ ]:
train_data = load_data()
theme_wise_data = load_theme_wise_data(train_data)

Number of examples = 75056
	Answerable questions = 50126
	Non-Answerable questions = 24930

Examples:
 | Theme  |  Paragraph...  |  Question | Answer_possible | Answer_text | Answer_start
1430 | Frédéric_Chopin  |  Some modern commenta...  |  Who said Chopin's works were modeled after Bach, Beethoven, Schubert and Field? | True | ['Richard Taruskin'] | [543]
2196 | The_Legend_of_Zelda:_Twilight_Princess  |  Twilight Princess ta...  |  Who releases Bulbins from the Realm of Twilight? | False | [] | []

Total 361 themes present.


## Creating list of questions for a particular theme

In [ ]:
def ques_ans(theme):
  t_ques = list(theme_wise_data[theme]['ques'])
  t_ans = list(theme_wise_data[theme]['ans'])
  t_ans = [i[2][2:-2] for i in t_ans]
  t = [[i, j] for i,j in zip(t_ques , t_ans )]
  new_t = {idx:[t[idx][0],t[idx][1]] for idx in range(len(t))}
  return t_ques, new_t

## Creating Embeddings and similarity scores

In [ ]:
def embed_ques(t_ques):
  encoded_data=model.signatures['question_encoder'](tf.constant(t_ques))['outputs']
  encoded_data=encoded_data.numpy()
  encoded_data = np.array(encoded_data)
  index =create_index_annoy(encoded_data)
  return index

In [ ]:
def create_index_annoy(embeddings, vector_length = 512, metric = 'angular', num_trees = 100):
  annoy_index = annoy.AnnoyIndex(vector_length, metric=metric)
  for i in range(len(embeddings)):
    annoy_index.add_item(i, embeddings[i])
  annoy_index.build(n_trees = num_trees)
  return annoy_index

In [ ]:
def top_k_ques(query):
    query_vector = model.signatures['question_encoder'](tf.constant(query))['outputs']
    query_vector =  query_vector.numpy()
    results = find_similar_annoy(index,query_vector[0])
    for result in results:
        print('\t',result)

## Finding the next k similar questions for a given query 

In [ ]:
def find_similar_annoy(index ,embedding, num_matches=5):
  '''Finds similar items to a given embedding in the ANN index'''
  
  ids = index.get_nns_by_vector(
  embedding, num_matches, search_k=-1, include_distances=True)
  # results =  [fetch_question_info(idx) for idx in ids]
  # fetch_q/uestion_info(ids)
  score = ids[1]
  questions = [new_t[id][0] for id in ids[0]]
  return [(a[0], a[1]) for a in zip(questions, score)]

In [ ]:
def print_sim_query(query, index):
  query_vector = model.signatures['question_encoder'](tf.constant(query))['outputs']
  query_vector =  query_vector.numpy()
  results = find_similar_annoy(index,query_vector[0])

  print("\n")
  for result in results:
      print('\t',result)

# Theme 1 - Beyonce

In [ ]:
theme = 'Beyoncé'

In [ ]:
t_ques, new_t = ques_ans(theme)

In [ ]:
index = embed_ques(t_ques)

## Analysis

In [ ]:
query= ["What was the name of Beyoncé's last solo album?"]
print_sim_query(query, index)



	 ("What was the name of Beyoncé's second solo album?", 0.2723705768585205)
	 ("What was the name of Beyoncé's first solo album?", 0.2802713215351105)
	 ("What is the name of Beyoncé's second album?", 0.4083942174911499)
	 ("What is the name of Beyoncé's fourth studio album?", 0.5114314556121826)
	 ('What solo album did Beyonce release in 2003?', 0.602519690990448)


In [ ]:
query= ["How has this foundation not changed in recent years"]
print_sim_query(query, index)



	 ('How has this foundation changed in recent years?', 0.3281075656414032)
	 ('Who inspired this hiatus?', 0.8138259649276733)
	 ('What do other critics claim?', 0.8885858058929443)
	 ('The Demand a Plan video campaign followed what tragic event?', 0.9217267036437988)
	 ("Who's death caused this protest?", 0.9632421731948853)


In [ ]:
query= ["What platform for crowdfunding was used in the concert?"]
print_sim_query(query, index)



	 ('What crowdfunding platform was used in the concert?', 0.23780225217342377)
	 ('What kind of platform was the song released?', 0.8722350001335144)
	 ('What had to be set in developing funding?', 1.05350661277771)
	 ('On June 1 , 2013 where was the concert held for "a Chime for Change"?', 1.0759516954421997)
	 ('What campaign did she contribute to?', 1.0950654745101929)


In [ ]:
query= ["What criteria have to be specified while developing funding?"]
print_sim_query(query, index)



	 ('What had to be set in developing funding?', 0.753717303276062)
	 ('What crowdfunding platform was used in the concert?', 1.1570402383804321)
	 ('What did she agree to do for 50 million dollars in 2012?', 1.2295639514923096)
	 ('What campaign did she contribute to?', 1.2316741943359375)
	 ('What does she get credits for in her music?', 1.2413731813430786)


In [ ]:
query= ["In 2005, what did Beyonce and Rowland find?"]
print_sim_query(query, index)



	 ('What did Beyonce and Rowland found in 2005?', 0.26844704151153564)
	 ('Who did Beyoncé and Parkwood Entertainment partner with in October 2014?', 0.8748180866241455)
	 ('Jay Z and Beyonce attended which event together in August of 2011?', 0.8915486335754395)
	 ('What type of accessory company did Beyonce partner with in 2005?', 0.9225267171859741)
	 ('What did Beyonce sign in 2013?', 0.9349269866943359)


In [ ]:
query= ["With what kind of accessory business did Beyonce collaborate in 2005?"]
print_sim_query(query, index)



	 ('What type of accessory company did Beyonce partner with in 2005?', 0.4395388662815094)
	 ('What company did Beyoncé get together with in 2005 to add shoes to her fashions?', 0.7291938662528992)
	 ('After her second solo album, what other entertainment venture did Beyonce explore?', 0.7702184915542603)
	 ("Beyonce's mother worked in what industry?", 0.8038763999938965)
	 ('What did Beyonce sign in 2013?', 0.8232850432395935)


In [ ]:
query= ["What percentage of the new business is owned by Beyonce?"]
print_sim_query(query, index)



	 ("What is Beyonce's percentage of ownership in the new venture?", 0.46717506647109985)
	 ('Totaling worldwide, how many records as Beyonce sold?', 0.7950239777565002)
	 ('How many records has Beyoncé sold in the United States?', 0.8102226853370667)
	 ('How many records has Beyonce sold in her 19 year career?', 0.8104107975959778)
	 ('How many records has Beyoncé sold throughout the world?', 0.8281041383743286)


# Theme 2 - Frédéric_Chopin

In [ ]:
theme = 'Frédéric_Chopin'

In [ ]:
t_ques, new_t = ques_ans(theme)

In [ ]:
index = embed_ques(t_ques)

## Analysis

In [ ]:
query= ["How much did Chopin's monument and funeral cost?"]
print_sim_query(query, index)



	 ("How much did Chopin's funeral and monument cost?", 0.081990547478199)
	 ("How much did Chopin's funeral cost?", 0.37547066807746887)
	 ("Where was Chopin's funeral held?", 0.8660853505134583)
	 ("How long was Chopin's funeral delayed?", 0.8949658274650574)
	 ('Which cemetery was Chopin buried in?', 0.9271509647369385)


In [ ]:
query= ["By how long was Chopin's funeral delayed?"]
print_sim_query(query, index)



	 ("How long was Chopin's funeral delayed?", 0.13632582128047943)
	 ("Where was Chopin's funeral held?", 0.8291794061660767)
	 ("How much did Chopin's funeral cost?", 0.835708737373352)
	 ("Who was the organist at Chopin's funeral?", 0.8773708343505859)
	 ("What song was sung at Chopin's funeral?", 0.8781919479370117)


In [ ]:
query= ["Who played the organ at Chopin's funeral?"]
print_sim_query(query, index)



	 ("Who was the organist at Chopin's funeral?", 0.4385497272014618)
	 ('At whose funeral did Chopin play in 1839?', 0.6213280558586121)
	 ("Who led Chopin's funeral procession?", 0.6579896807670593)
	 ("What song was sung at Chopin's funeral?", 0.675342321395874)
	 ("What instrument did Chopin play at  Adolphe Nourrit's funeral?", 0.6897367238998413)


In [ ]:
query= ["Which sonata is worthy of Brahms as per historians?"]
print_sim_query(query, index)



	 ('According to music historians, which sonata is similar to German tradition and worthy of Brahms?', 0.7455739974975586)
	 ("What piece of Bach's did Chopin take inspiration for his preludes?", 0.8156876564025879)
	 ('What piece did Chopin dedicate to Schumann?', 0.8419543504714966)
	 ('What piece did Chopin dedicate to Liszt?', 0.8464268445968628)
	 ('What piece did Chopin compose in 1844?', 0.8759956955909729)


In [ ]:
query= ["When claiming Chopin's nationalism was overrated, what 2 dancers were referenced ? "]
print_sim_query(query, index)



	 ("George Golos references what two musicians when claiming Chopin's nationalism was overrated? ", 0.6044313311576843)
	 ("Who were two of Chopin's contemporaries?", 0.8973211646080017)
	 ("Who choreographed a ballet which included Chopin's work?", 0.9447776675224304)
	 ("Which two of Chopin's friends became part of his Paris milieu?", 0.9582419991493225)
	 ("What three composers influenced Chopin's work?", 0.9607998132705688)


In [ ]:
query= [" Who danced which included Chopin's work? "]
print_sim_query(query, index)



	 ("Who choreographed a ballet which included Chopin's work?", 0.5014089941978455)
	 ("Who played the earlier known recording of Chopin's work?", 0.7038514018058777)
	 ("What is the name of the ballet that included Chopin's work?", 0.7139905691146851)
	 ("Who released the first collection of Chopin's works?", 0.7374522686004639)
	 ("Which 1909 ballet used Chopin's music?", 0.7641186714172363)


# Theme 3 - New_York_City

In [ ]:
theme = 'New_York_City'

In [ ]:
t_ques, new_t = ques_ans(theme)

In [ ]:
index = embed_ques(t_ques)

## Analysis

In [ ]:
query= ["What was year when New York adopted the mayor?"]
print_sim_query(query, index)



	 ('In what year did New York City adopt the mayor-council form of government?', 0.7314983010292053)
	 ('When was New York City established?', 0.7921110391616821)
	 ('In what year did New York become the largest city in the United States?', 0.7946471571922302)
	 ('By which year, did New York City become the largest city in the United States?', 0.8021202087402344)
	 ('In what year did New York become the United States capital?', 0.8064650893211365)


In [ ]:
query= ["By what time did New York become the capital"]
print_sim_query(query, index)



	 ('In what year did New York become the United States capital?', 0.4954143464565277)
	 ('In what year did New York cease the be the capital of the United States?', 0.5808302164077759)
	 ('When was New York City established?', 0.6266717910766602)
	 ('In what year did New York become the largest city in the United States?', 0.7234587669372559)
	 ('By which year, did New York City become the largest city in the United States?', 0.7274383902549744)


In [ ]:
query= ["By what time New York was created"]
print_sim_query(query, index)



	 ('When was New York City established?', 0.5855913162231445)
	 ('The English occupied New York City beginning on what date?', 0.776697039604187)
	 ('In what year did New York become the United States capital?', 0.7908604741096497)
	 ('In what year did the first European arrive in the New York area?', 0.7914308309555054)
	 ('In what year did New York become the largest city in the United States?', 0.8113515377044678)


In [ ]:
query= ["What year did New York surpass Los Angeles as the country's biggest city?"]
print_sim_query(query, index)



	 ('In what year did New York become the largest city in the United States?', 0.4836888611316681)
	 ('By which year, did New York City become the largest city in the United States?', 0.48466891050338745)
	 ('New York City is the biggest city in the United States since what historical date?', 0.6833901405334473)
	 ('In what year did the population of New York first reach an all-time high in this period?', 0.7013521790504456)
	 ('In what year did New York become the United States capital?', 0.7431342005729675)


In [ ]:
query= ["When did New York's population initially hit an all-time high during this time period?"]
print_sim_query(query, index)



	 ('In what year did the population of New York first reach an all-time high in this period?', 0.4841434955596924)
	 ('In what year was New York ranked first on the Global Economic Power Index?', 0.7494376301765442)
	 ('In what year did New York become the largest city in the United States?', 0.7518388032913208)
	 ('By which year, did New York City become the largest city in the United States?', 0.7715208530426025)
	 ('In the 1930s, New York City had more than 10 million people becoming the first what in history?', 0.793397843837738)


In [ ]:
query= ["What is the number of track lines are there on New York's commuter network?"]
print_sim_query(query, index)



	 ("How many rail lines are there on New York City's commuter rail network?", 0.6007077097892761)
	 ("About how many stations does New York City's commuter rail network have?", 0.6621059775352478)
	 ('How many daily bicycle riders are there in NYC?', 0.8890556693077087)
	 ('How many people rode the New York City Subway in 2014?', 0.89009028673172)
	 ('How many boroughs does New York City have?', 0.9176472425460815)


# Theme 4 - Kanye_West

In [ ]:
theme = 'Kanye_West'

In [ ]:
t_ques, new_t = ques_ans(theme)

In [ ]:
index = embed_ques(t_ques)

##Analysis

In [ ]:
query= ['What university prompted Wests mother to relocate the family to Asia?']
print_sim_query(query, index)



	 ("What University was the cause of West's mother moving the family to China?", 0.4612826406955719)
	 ('What school awarded Kanye west an honorary doctorate?', 1.0234363079071045)
	 ('What school did Kanye West go to in 1997?', 1.0398316383361816)
	 ("Where did Kanye West's mother work before becoming his manager?", 1.0499783754348755)
	 ("What was the name of Kanye West's high school?", 1.0578633546829224)


In [ ]:
query= ['Before becoming Kanye Wests assistant, where did his mother work?']
print_sim_query(query, index)



	 ("Where did Kanye West's mother work before becoming his manager?", 0.41114863753318787)
	 ("What was Kanye West's first job title after art school?", 0.7896179556846619)
	 ('Where did Kanye receive a scholarship to attend?', 0.8363249897956848)
	 ('What kind of background did Kanye West grow up in?', 0.8681840300559998)
	 ('Where did Kanye West live as a child?', 0.8720035552978516)


In [ ]:
query= ['What type of upbringing did Kanye West have?']
print_sim_query(query, index)



	 ('What kind of background did Kanye West grow up in?', 0.619104266166687)
	 ('What type of people did Kanye state had more power than Black people?', 0.8380099534988403)
	 ('What kind of music does Kanye West perform?', 0.838288426399231)
	 ('How well did Kanye do in high school?', 0.8466695547103882)
	 ("What did Kanye's peers see him as while he tried to pursue his rap career?", 0.855300784111023)


In [ ]:
query= ['According to Kanye, what type of people has greater power than Black people?']
print_sim_query(query, index)



	 ('What type of people did Kanye state had more power than Black people?', 0.4698088765144348)
	 ("Who did Kanye West say doesn't care about black people?", 0.8581160306930542)
	 ("What did Kanye's peers see him as while he tried to pursue his rap career?", 0.9881647229194641)
	 ('Although Kanye was achieving fame by producing, what did he actually want to be?', 0.988705575466156)
	 ("What person was Kanye's rant on Twitter directed at?", 1.0002437829971313)


In [ ]:
query= ['In 2001, Kanye was given producer credit on which album?']
print_sim_query(query, index)



	 ('What album did Kanye receive production credits on in 2001?', 0.4002895653247833)
	 ("What was the name of Kanye's second studio album?", 0.6303995847702026)
	 ('What was the name of the joint album Kanye made with Jay Z?', 0.6521729230880737)
	 ("What was the name of Kanye's third album?", 0.6539018750190735)
	 ("What was the title of Kanye's sixth album?", 0.6585341095924377)


# Theme 5 - Buddhism

In [ ]:
theme = 'Buddhism'

In [ ]:
t_ques, new_t = ques_ans(theme)

In [ ]:
index = embed_ques(t_ques)

## Analysis

In [ ]:
query= ['Which branch follows the Zen teachings?']
print_sim_query(query, index)



	 ('Which branch practices the teachings of Zen?', 0.38213881850242615)
	 ('What minor branch is sometimes attributed to Mahayana?', 0.8329461216926575)
	 ('Which schools of Zen likes the use of meditation on the koan for spiritual breakthroughs?', 0.8493402004241943)
	 ('What is sometimes viewed as the third branch of Buddhism?', 0.9110625982284546)
	 ('In which branch of Buddhism is it believed that there can be no divine salvation or forgiveness for karma?', 0.9205144047737122)


In [ ]:
query= ['Which school of Buddhism holds the view that karma cannot be forgiven or saved by God?']
print_sim_query(query, index)



	 ('In which branch of Buddhism is it believed that there can be no divine salvation or forgiveness for karma?', 0.5482146143913269)
	 ('What actions does karma refer to in Buddhism?', 0.8524971604347229)
	 ('What is karma according to Buddhism?', 0.8535218834877014)
	 ('What branch of Buddhism rejects that there is a transitional state between lives?', 0.8561480045318604)
	 ('What form of Buddhism has declined to the point where few are capable of following the path?', 0.8710591197013855)


In [ ]:
query= ['What branch of Buddhism has become so rare that few people can still pursue the path?']
print_sim_query(query, index)



	 ('What form of Buddhism has declined to the point where few are capable of following the path?', 0.6057392358779907)
	 ('What is sometimes viewed as the third branch of Buddhism?', 0.7603534460067749)
	 ('In which branch of Buddhism is it believed that there can be no divine salvation or forgiveness for karma?', 0.7651271820068359)
	 ('What branch of Buddhism rejects that there is a transitional state between lives?', 0.7882188558578491)
	 ('What is the most widely practised Eastern Buddhism?', 0.8200209140777588)


In [ ]:
query= ['What is regarded as the third tree of Buddhism occasionally?']
print_sim_query(query, index)



	 ('What is sometimes viewed as the third branch of Buddhism?', 0.5807662606239319)
	 ('What was the tree renamed that Gautama achieved enlightenment under?', 0.7413508296012878)
	 ('What is the most widely practised Eastern Buddhism?', 0.8034606575965881)
	 ('When he was 35 Gautama sat in meditation under what tree?', 0.8126512765884399)
	 ('What is another name for Nikaya buddhism?', 0.8138707876205444)


In [ ]:
query= ['What is the name of the middle ground between indulgence and mortification?']
print_sim_query(query, index)



	 ('What is the path of moderation between the extremes of self-indulgence and self-mortification called?', 0.8737300038337708)
	 ('What is the avoidance of bad acts called?', 1.0166782140731812)
	 ('What is another name for Nikaya buddhism?', 1.0379949808120728)
	 ('Avoidance of unwholesome actions and use of positive actions is called what?', 1.057279348373413)
	 ('What is Esoteric Buddhism also called?', 1.0651354789733887)


#Theme 6 - ASCII

In [ ]:
theme = 'ASCII'

In [ ]:
t_ques, new_t = ques_ans(theme)

In [ ]:
index = embed_ques(t_ques)

##Analysis

In [ ]:
query= ['What was the name of the signal that alerted the sender of an imminent overflow?']
print_sim_query(query, index)



	 ('What was the warning signal called that warned the sender that there was impending overflow?', 0.5235121250152588)
	 ('What was the warning signal that warned TAPE that there was impending overflow?', 0.814116358757019)
	 ('What is impending overflow replaced by on some systems?', 1.0514439344406128)
	 ('What was the code first called in Europe?', 1.1051982641220093)
	 ('What did older operating systems use to mark the end of the text?', 1.140036940574646)


In [ ]:
query= ['What signalled the end of the text in earlier operating systems?']
print_sim_query(query, index)



	 ('What did older operating systems use to mark the end of the text?', 0.702467679977417)
	 ('What did older operating systems use to mark control code?', 0.9564542174339294)
	 ('What do computers using operating systems use for line endings?', 0.9871618747711182)
	 ('What interpretation became common in Unix systmes?', 0.9992682933807373)
	 ('What is using Z as the end of text analogous to?', 1.0113563537597656)


In [ ]:
query= ['Where the typebars hit the ribbon, what moves the paper?']
print_sim_query(query, index)



	 ('Which advances the paper where the typebars strike the ribbon?', 0.6677637696266174)
	 ('What casued the automatic paper tape reader to stop?', 1.074441909790039)
	 ('When was paper tape popular?', 1.130643606185913)
	 ('Where were the letters DEC in position?', 1.1517703533172607)
	 ('What caused the automatic paper tape reader to start again?', 1.1538203954696655)


In [ ]:
query= ['What were the two codes that teletype machines need to terminate a line?']
print_sim_query(query, index)



	 ('Teletype machines had to have two codes to termanate a line, what were they?', 0.7270897030830383)
	 ('What is another name for Teletype machines?', 0.9325006604194641)
	 ('What was code 127 labeled as for the Teletype?', 0.9456164836883545)
	 ('Why did Teletype have the code for "rubout"?', 0.9873180985450745)
	 ('How many specific characters are there in the ASCII Teletype machines?', 1.0240564346313477)


In [ ]:
query= ['When was the first significant code revision completed?']
print_sim_query(query, index)



	 ('When was the first major revision of the code done?', 0.3822709023952484)
	 ('What were code points ultimately published as?', 0.9067711234092712)
	 ('When was the introduction of IBM?', 0.9538112282752991)
	 ('When did work on the ASCII standard begin? ', 0.9592514634132385)
	 ('When was ASCII first commercially used?', 0.9922497868537903)


In [ ]:
query= ['What kinds of extensions do these other standards organisations have?']
print_sim_query(query, index)



	 ('What type of extensions do these other standard bodies have?', 0.6129148602485657)
	 ('What type of extensions do these other character encodings have?', 0.8505483865737915)
	 ('ASCII was incorporated into what other extensions?', 1.0342093706130981)
	 ('Why are the other versions of ASCII charging extra for extensions?', 1.063830018043518)
	 ('What was incorporated into the Unicode extensions?', 1.0809438228607178)


#Theme 7 - Republic_of_the_Congo

In [ ]:
theme= 'Republic_of_the_Congo'

In [ ]:
t_ques, new_t = ques_ans(theme)

In [ ]:
index = embed_ques(t_ques)

##Analysis

In [ ]:
query= ['What caused the Bantu cultures rise to dominance in the late years?']
print_sim_query(query, index)



	 ('What was responsible for the power of Bantu societies growing in the late 19th century?', 0.6028746366500854)
	 ('What did European traders exchange with Bantu people?', 0.8925159573554993)
	 ('When did the Pygmy people displace the Bantu?', 0.8964157104492188)
	 ('When did Bantu tribes arrive in the area formerly populated by the Pygmy people?', 0.9157742261886597)
	 ('What area did the Bantu colonize in the late 19th century?', 0.9159032702445984)


# Theme 8 - Madonna_(entertainer)

In [ ]:
theme = 'Madonna_(entertainer)'

In [ ]:
t_ques, new_t = ques_ans(theme)

In [ ]:
index = embed_ques(t_ques)

##Analysis

In [ ]:
query= ['What negative effects did Madonnas portrayal of blasphemy in her newest video, Like A Prayer, cause?']
print_sim_query(query, index)



	 ("With Madonna's new video, Like A Prayer, the blasphemy that she portrayed in the video led to what consequences?", 0.7037410140037537)
	 ("What was Madonna's mother's reaction to her questioning about her illness?", 0.9074692130088806)
	 ("Name one of Madonna's major influence?", 0.9358859658241272)
	 ('Which religion was Madonna dedicated to?', 0.9584553837776184)
	 ('Madonna was born to which religion?', 0.9810348153114319)


In [ ]:
query= ['What is the name of the art piece that Madonna and fashion photographer Chris Klein collaborated on for an exhibition?']
print_sim_query(query, index)



	 ('What is the name of the exhibition installation that Madonna teamed up with fashion photographer Chris Klein called?', 0.5428701639175415)
	 ('What is the name of the clothing line Madonna released with her daughter?', 0.8831934928894043)
	 ("Who is the stylist and jewellery designer that created Madonna's look?", 0.9169471263885498)
	 ("Which 2 album show Madonna's artistic statement?", 0.9196118712425232)
	 ("What is Madonna's compilation album called?", 0.9316215515136719)


In [ ]:
query= ['Who has enjoyed the greatest level of unary success in the history of the American singles chart?']
print_sim_query(query, index)



	 ('Who is the most successful solo artist in the history of American singles chart?', 0.8740720748901367)
	 ('Who listed Ray of Light as "The 500 Greatest Albums of All Time?"', 1.0533597469329834)
	 ('Who has the most number one songs in an active BiIlboard chart?', 1.0803903341293335)
	 ('American Life reached which number at the Hot 100?', 1.1259734630584717)
	 ('Madonna surpassed which artist with the most top-ten hits?', 1.1348237991333008)


In [ ]:
query= ['Which record has Kabbalistic influences?']
print_sim_query(query, index)



	 ('The influence of Kabbalah is seen in which album?', 0.9498804807662964)
	 ('Which award the the album win?', 1.06398344039917)
	 ('Which 2 albums explored the New Lack Swing?', 1.0951555967330933)
	 ('True Blue has what type of music Incorporated in it?', 1.1013261079788208)
	 ('Which edition of the Guinness Book of World Record states that no female artist sold more records than Madonna?', 1.101597547531128)


In [ ]:
query= ['Who said that Madonna overstated the extent of her charitable contributions?']
print_sim_query(query, index)



	 ("Who criticized Madonna's charity efforts, stating that she exaggerated her contributions?", 0.518703818321228)
	 ('Who said that Madonna changed the world social history?', 0.7492490410804749)
	 ('Who said that Madonna is like post-moderm art?', 0.8066521286964417)
	 ('Who noted that Madonna was reversing the gender role?', 0.8197821974754333)
	 ('Who wrote that Madonna is opportunistic, manipulative and ruthless?', 0.8361309170722961)


#Theme 9 - Edmund_Burke

In [ ]:
theme = 'Edmund_Burke'

In [ ]:
t_ques, new_t = ques_ans(theme)

In [ ]:
index = embed_ques(t_ques)

##Analysis

In [ ]:
query= ['Burke might not have continued his history of England if another authors was written before his by whom?']
print_sim_query(query, index)



	 ("Which author's history of England being published before Burke's might have dissuaded Burke from continuing his?", 0.5110180377960205)
	 ('Whose biography did Burke write?', 0.8000097274780273)
	 ('Who wrote a biography of Burke?', 0.8035860657691956)
	 ('Who wrote to Burke on Nov 4, 1793?', 0.8314825296401978)
	 ('Who did Burke write to on Nov 4, 1793?', 0.8400540351867676)


In [ ]:
query= ['Opposite to which state burke thought that backing the war against the mutiny was an act ?']
print_sim_query(query, index)



	 ('Burke considered supporting the war against the revolution as being against which nation?', 0.7691667079925537)
	 ('Who did Burke think the war was against, rather than France as a whole?', 0.8651604652404785)
	 ('Burke thought the French Revolution was against whose principles?', 0.8761308193206787)
	 ('What faction was Burke opposed to in La Vendée?', 0.8968031406402588)
	 ('Who did Burke want to support the revolution?', 0.9082686901092529)


In [ ]:
query= ['What insight into human nature did Rousseau claim Burke possessed?']
print_sim_query(query, index)



	 ('What did Rousseau say Burke had in regards to understanding human nature?', 0.48615971207618713)
	 ('What did Burke think was the foundation of society?', 0.8569153547286987)
	 ('What did Burke think was crucial for human life?', 0.862197756767273)
	 ('What did Burke believe was against the natural order of events?', 0.8782197833061218)
	 ('What did Burke think a social hierarchy should be based on?', 0.8886898756027222)


In [ ]:
query= ['Who believed Burke to be like an architect renovating an old home as opposed to demolishing it?']
print_sim_query(query, index)



	 ('Who thought Burke was like an architect remodeling an old house instead of tearing it down?', 0.5309524536132812)
	 ('Whose ascendancy did Burke think was malignant?', 0.8425593376159668)
	 ('Who did Burke think protected unpopular minorities?', 0.8745482563972473)
	 ('Who did Burke think a social class structure benefited?', 0.8815036416053772)
	 ('Who did Burke say we look to with reverence?', 0.8883112668991089)


#Theme 10 - Adolescence 

In [ ]:
theme = 'Adolescence'

##Analysis

In [ ]:
query = ['When are a number of changes triggered due to increased hormone production ']
top_k_ques(query)

	 ('How often could paymasters make withdrawals?', 1.2053004503250122)
	 ('When did Mary Wollstonecraft publish her response?', 1.2379944324493408)
	 ("When was the tenth printing of the French translation of 'Reflections on the Revolution in France'?", 1.2449085712432861)
	 ('When did Richard resign?', 1.250648021697998)
	 ('When did Wordsworth initially attack Burke?', 1.2516568899154663)


In [ ]:
query = ['At what stage of life does a decrease in hormone production trigger a number of physical changes?']
top_k_ques(query)

	 ('How often could paymasters make withdrawals?', 1.2344247102737427)
	 ('When did Richard resign?', 1.242231845855713)
	 ('When did Jean-Jacques Rousseau die?', 1.2619874477386475)
	 ('When did Hamilton die?', 1.268661618232727)
	 ('When did Wordsworth initially attack Burke?', 1.274701714515686)


In [ ]:
query = ['What organ produces testosterone in males?']
top_k_ques(query)

	 ("Who wrote 'The Rights of Man'?", 1.2285473346710205)
	 ("Who wrote 'A Vindication of the Rights of Men'?", 1.2597192525863647)
	 ('What geographical feature would prevent the Americans from retreating?', 1.2623785734176636)
	 ("What was William Gerard Hamilton's nickname?", 1.2701637744903564)
	 ('What was Burke a member of?', 1.2755508422851562)


In [ ]:
query = ['Which organ in the male body does not release testosterone?']
top_k_ques(query)

	 ('What geographical feature would prevent the Americans from retreating?', 1.217445731163025)
	 ("Who wrote 'The Rights of Man'?", 1.2549430131912231)
	 ('Burke claimed that the contract did not affect what groups of people?', 1.258540153503418)
	 ('What city did Burke represent in parliament?', 1.2600822448730469)
	 ('What nickname was Burke given for his emotional indignation?', 1.2663604021072388)


In [ ]:
query = ['What organ produces estrogen in females?']
top_k_ques(query)

	 ("Who wrote 'The Rights of Man'?", 1.2703571319580078)
	 ('What geographical feature would prevent the Americans from retreating?', 1.2735867500305176)
	 ('Which administration proposed the Paymaster act?', 1.2749035358428955)
	 ('Who did Parisian women want to return to Paris?', 1.2784361839294434)
	 ('When did Mary Wollstonecraft publish her response?', 1.2837767601013184)


In [ ]:
query = ['Which organ in the female body does not release estrogen?']
top_k_ques(query)

	 ('What geographical feature would prevent the Americans from retreating?', 1.217729091644287)
	 ('Which administration proposed the Paymaster act?', 1.2699962854385376)
	 ('What area could the American colonists retreat to?', 1.2720457315444946)
	 ('What was Cobden an activist against?', 1.2721219062805176)
	 ('Burke claimed that the contract did not affect what groups of people?', 1.2763144969940186)


In [ ]:
query = ['What is the stage of life between childhood and adulthood called?']
top_k_ques(query)

	 ('What was the impeachment formally called?', 1.1795594692230225)
	 ('What was the name of the letters Young published?', 1.2132208347320557)
	 ("Burke's unfinished letter to Arthur Young became part of what publication?", 1.2485876083374023)
	 ('What did Burke call corporate tyranny in India?', 1.2774399518966675)
	 ('What was the subject of the letter Arthur Young did not complete?', 1.2836214303970337)


In [ ]:
query = ['What is the period between childhood and adulthood not viewed as?']
top_k_ques(query)

	 ('What was the subject of the letter Arthur Young did not complete?', 1.1907283067703247)
	 ("Burke's unfinished letter to Arthur Young became part of what publication?", 1.2042601108551025)
	 ('When did Mary Wollstonecraft publish her response?', 1.206414818763733)
	 ('What was founded the year before the contract?', 1.2104051113128662)
	 ('When was the Revolution Society founded?', 1.216540813446045)


In [ ]:
query = ['What physical changes occur during puberty besides physical maturation?']
top_k_ques(query)

	 ('What was published in 1775?', 1.2930465936660767)
	 ('Who converted into Catholicism?', 1.3011642694473267)
	 ('What did Burke note in the preface of the first edition of the book?', 1.3159685134887695)
	 ('Which victories against the Americans did the British celebrate?', 1.3161762952804565)
	 ('What did François-Louis-Thibault de Menonville publish in April 1791?', 1.317848563194275)


In [ ]:
query = ['What changes do not occur during puberty in addition to physical maturation?']
top_k_ques(query)

	 ('What was the subject of the letter Arthur Young did not complete?', 1.2729597091674805)
	 ('What did Burke note in the preface of the first edition of the book?', 1.281389832496643)
	 ('What was published in 1775?', 1.2860881090164185)
	 ('What geographical feature would prevent the Americans from retreating?', 1.2975492477416992)
	 ('What did François-Louis-Thibault de Menonville publish in April 1791?', 1.2985872030258179)


In [ ]:
query = ['why does the cardiovascular system change during puberty?']
top_k_ques(query)

	 ('When did Wordsworth initially attack Burke?', 1.2916131019592285)
	 ('Who converted into Catholicism?', 1.2917991876602173)
	 ('When was Trinity College Dublin established?', 1.309001088142395)
	 ('What was the name of the letters Young published?', 1.3103127479553223)
	 ('When was the Revolution Society founded?', 1.316306233406067)


In [ ]:
query = ['How do the cardiovascular system not change during puberty?']
top_k_ques(query)

	 ('When did Wordsworth initially attack Burke?', 1.2937309741973877)
	 ('How often could paymasters make withdrawals?', 1.2990998029708862)
	 ('Who converted into Catholicism?', 1.3046412467956543)
	 ('What was the name of the letters Young published?', 1.3097878694534302)
	 ('How did Burke say we look towards magistrates?', 1.3100730180740356)


In [ ]:
query = ['What changes to the body result from the increased size and capacity of the heart and lungs during puberty?']
top_k_ques(query)

	 ('What did Mackintosh claim he had more of than any other writer?', 1.282840371131897)
	 ('What was the name of the letters Young published?', 1.2836024761199951)
	 ('What was published in 1775?', 1.2848434448242188)
	 ("Burke's unfinished letter to Arthur Young became part of what publication?", 1.2887433767318726)
	 ('Who converted into Catholicism?', 1.2979382276535034)


In [ ]:
query = ['What changes to the body do not result from the decreased size and capacity of the heart and lungs during puberty?']
top_k_ques(query)

	 ('What was the subject of the letter Arthur Young did not complete?', 1.263871669769287)
	 ('What was the name of the letters Young published?', 1.276983380317688)
	 ('What did Mackintosh claim he had more of than any other writer?', 1.2808232307434082)
	 ('What geographical feature would prevent the Americans from retreating?', 1.2815642356872559)
	 ("Burke's unfinished letter to Arthur Young became part of what publication?", 1.282180905342102)


In [ ]:
query = ['Which gender has a higher red blood cell count on average?']
top_k_ques(query)

	 ("Who wrote 'A Vindication of the Rights of Men'?", 1.2770447731018066)
	 ('Who did Burke think Britain could beat at war?', 1.290177345275879)
	 ('Burke claimed that the contract did not affect what groups of people?', 1.3044127225875854)
	 ("Who wrote 'The Rights of Man'?", 1.3065577745437622)
	 ('Who did Burke think a social class structure benefited?', 1.3066191673278809)


In [ ]:
query = ['Which gender has a lower white blood cell count on average?']
top_k_ques(query)

	 ("Who wrote 'A Vindication of the Rights of Men'?", 1.2645127773284912)
	 ('Who did Burke think Britain could beat at war?', 1.2746580839157104)
	 ('Burke claimed that the contract did not affect what groups of people?', 1.278517246246338)
	 ("Who wrote 'The Rights of Man'?", 1.2884252071380615)
	 ("Who wrote Dodsley's biography?", 1.2886765003204346)


In [ ]:
query = ['Which body system do the lungs play a key role in?']
top_k_ques(query)

	 ('What geographical feature would prevent the Americans from retreating?', 1.2132680416107178)
	 ('What did Burke think was crucial for human life?', 1.2444883584976196)
	 ('Who did Burke think a social class structure benefited?', 1.2526333332061768)
	 ('What region did Prince Charles use to back up his points? ', 1.2638839483261108)
	 ('Whose principles did L. Bol imitate?', 1.2647424936294556)


In [ ]:
query = ['Which body system do the lungs not play a key role in?']
top_k_ques(query)

	 ('What geographical feature would prevent the Americans from retreating?', 1.1821203231811523)
	 ('What region did Prince Charles use to back up his points? ', 1.2548292875289917)
	 ('Burke claimed that the contract did not affect what groups of people?', 1.254873275756836)
	 ('What did Burke think was crucial for human life?', 1.2568868398666382)
	 ('Who did Burke think a social class structure benefited?', 1.2581617832183838)


In [ ]:
query = ['Is the brain fully developed at the onset of puberty?']
top_k_ques(query)

	 ('What was founded the year before the contract?', 1.2923957109451294)
	 ('Who converted into Catholicism?', 1.3013157844543457)
	 ('When was Trinity College Dublin established?', 1.3074853420257568)
	 ('What was the name of the letters Young published?', 1.3114265203475952)
	 ('When did Paymasters begin to get money directly from the HM treasury?', 1.3200222253799438)


In [ ]:
query = ['Is the brain not fully developed at the onset of puberty?']
top_k_ques(query)

	 ("Who said he 'perfectly differed from Mr. Fox'?", 1.2860392332077026)
	 ('What was the name of the letters Young published?', 1.3081443309783936)
	 ('Who converted into Catholicism?', 1.3223891258239746)
	 ('What was founded the year before the contract?', 1.323432445526123)
	 ('When did Paymasters begin to get money directly from the HM treasury?', 1.3255380392074585)


In [ ]:
query = ['At what age is the brain considered to be approximately 90% of its adult size?']
top_k_ques(query)

	 ('When was Trinity College Dublin established?', 1.2446255683898926)
	 ('When did Edmund Burke graduate?', 1.2508925199508667)
	 ('When did Burke begin attending Trinity College Dublin?', 1.2610880136489868)
	 ('When did an angel supposedly speak to Bathurst?', 1.2614587545394897)
	 ('When did Burke become the chief editor?', 1.276320457458496)


In [ ]:
query = ['The brain reaches what percentage of its adult size by the time a person is 60 years old?']
top_k_ques(query)

	 ('How long was the book Robert Dodsley wrote?', 1.2878847122192383)
	 ("How many words long was Burke's history of England contracted to be?", 1.2912001609802246)
	 ('When did Jean-Jacques Rousseau die?', 1.291517972946167)
	 ("How many pages long was Burke's history of England contracted to be?", 1.2916827201843262)
	 ("How many words was Lord Acton's commentary?", 1.303222894668579)


In [ ]:
query = ['What areas of the brain undergo the most significant changes during puberty?']
top_k_ques(query)

	 ("Who was featured in the most well-known part of Burke's Reflections?", 1.2764241695404053)
	 ('What region did Prince Charles use to back up his points? ', 1.2862557172775269)
	 ('What did Burke think was crucial for human life?', 1.2870371341705322)
	 ("Burke's unfinished letter to Arthur Young became part of what publication?", 1.2876253128051758)
	 ('What geographical feature would prevent the Americans from retreating?', 1.28914213180542)


In [ ]:
query = ['The smallest changes in the brain during puberty occur in the parts of the cortex that process what kinds of information?']
top_k_ques(query)

	 ("Burke's unfinished letter to Arthur Young became part of what publication?", 1.221121072769165)
	 ('What did Rousseau say Burke had in regards to understanding human nature?', 1.2450579404830933)
	 ('What was the name of the letters Young published?', 1.2518717050552368)
	 ('What was the subject of the letter Arthur Young did not complete?', 1.2550405263900757)
	 ('What publication did Burke found?', 1.2567905187606812)


In [ ]:
query = ['Which regions of the brain continue to mature and develop during the adolescent years?']
top_k_ques(query)

	 ('What region did Prince Charles use to back up his points? ', 1.2163207530975342)
	 ('What geographical feature would prevent the Americans from retreating?', 1.2329826354980469)
	 ("Burke's unfinished letter to Arthur Young became part of what publication?", 1.2442278861999512)
	 ('What was the name of the letters Young published?', 1.2576898336410522)
	 ('Who overcame his emotions in order to finish his speech?', 1.2588074207305908)


In [ ]:
query = ['What parts of the brain become less complex into the late teens?']
top_k_ques(query)

	 ('What was the name of the letters Young published?', 1.2656153440475464)
	 ('What did Mackintosh claim he had more of than any other writer?', 1.2707489728927612)
	 ("What did Francis think Burke's writing had too much of?", 1.2744094133377075)
	 ("Burke's unfinished letter to Arthur Young became part of what publication?", 1.2826820611953735)
	 ('Who overcame his emotions in order to finish his speech?', 1.289731502532959)


In [ ]:
query = ['What are the two neurotransmitters that play a role in increased emotional reactivity and sensitivity to rewards and stress during adolescence?']
top_k_ques(query)

	 ('What was the name of the letters Young published?', 1.2492859363555908)
	 ('Which administration proposed the Paymaster act?', 1.2548224925994873)
	 ('What nickname was Burke given for his emotional indignation?', 1.259202480316162)
	 ('Who overcame his emotions in order to finish his speech?', 1.2645432949066162)
	 ('What did Rousseau say Burke had in regards to understanding human nature?', 1.2698105573654175)


In [ ]:
query = ['Which two neurotransmitters cause adolescents to be less emotional and less responsive to rewards and stresses?']
top_k_ques(query)

	 ('Which administration proposed the Paymaster act?', 1.247583270072937)
	 ('Who overcame his emotions in order to finish his speech?', 1.249194622039795)
	 ('What was the name of the letters Young published?', 1.2492200136184692)
	 ('What did Rousseau say Burke had in regards to understanding human nature?', 1.273706316947937)
	 ("Burke's unfinished letter to Arthur Young became part of what publication?", 1.278310775756836)


In [ ]:
query = ['What type of cognitive process is associated with the development of deductive reasoning?']
top_k_ques(query)

	 ('What type of doctrine did Burke propose?', 1.1680338382720947)
	 ('What did Burke think a social hierarchy should be based on?', 1.1688759326934814)
	 ('What did Rousseau say Burke had in regards to understanding human nature?', 1.176338791847229)
	 ('What philosophy did Price support?', 1.1851774454116821)
	 ('What did Burke believe was against the natural order of events?', 1.1935421228408813)


In [ ]:
query = ['Inductive reasoning leads to the development of what type of thinking?']
top_k_ques(query)

	 ('What did Burke think a social hierarchy should be based on?', 1.1677690744400024)
	 ('What philosophy did Price support?', 1.1684865951538086)
	 ('What type of doctrine did Burke propose?', 1.1690744161605835)
	 ('What did Burke believe was against the natural order of events?', 1.1744353771209717)
	 ('Whose principles did Burke imitate?', 1.1752909421920776)


In [ ]:
query = ['What are some of the perspectives of adolescence in society?']
top_k_ques(query)

	 ('What did Rousseau say Burke had in regards to understanding human nature?', 1.1795889139175415)
	 ('What did Burke think a social hierarchy should be based on?', 1.2026410102844238)
	 ('What did Burke think was the foundation of society?', 1.241262435913086)
	 ("What did Lord Chesterfield believe Bolingbroke's arguments could also apply to?", 1.2435874938964844)
	 ('What philosophy did Price support?', 1.2530182600021362)


In [ ]:
query = ['What is adolescence viewed as?']
top_k_ques(query)

	 ('What was the name of the letters Young published?', 1.2361997365951538)
	 ('What did Rousseau say Burke had in regards to understanding human nature?', 1.2746394872665405)
	 ('What did Paine call Reflections?', 1.2819792032241821)
	 ('What was the subject of the letter Arthur Young did not complete?', 1.287163257598877)
	 ('What was Cobden an activist against?', 1.292448878288269)


In [ ]:
query = ['What is adolescence a period of?']
top_k_ques(query)

	 ("How long before Hastings's trial had Britain been dealing with Indian unrest?", 1.2326998710632324)
	 ('What was the subject of the letter Arthur Young did not complete?', 1.2545785903930664)
	 ('When was the Revolution Society founded?', 1.2587389945983887)
	 ('What was the impeachment formally called?', 1.2650803327560425)
	 ('When was Trinity College Dublin established?', 1.2682859897613525)


In [ ]:
query = ['Adolescents thinking is less bound to what than that of children?']
top_k_ques(query)

	 ('Who wanted the argument to go public?', 0.8725441694259644)
	 ("What did Burke say 'envenoms everything it touches'?", 0.9980752468109131)
	 ("Who said he 'perfectly differed from Mr. Fox'?", 1.042303204536438)
	 ('Who did Price think should see themselves as citizens of the world?', 1.080922245979309)
	 ("Who thought Burke and Fox shouldn't have made their argument public?", 1.1321946382522583)


In [ ]:
query = ['Adolescents become what?']
top_k_ques(query)

	 ('Who wanted the argument to go public?', 0.7257106900215149)
	 ("What did Burke say 'envenoms everything it touches'?", 0.9964730739593506)
	 ("Who said he 'perfectly differed from Mr. Fox'?", 1.0520625114440918)
	 ('What did Charles James Fox translate?', 1.1280627250671387)
	 ('Who did Price think should see themselves as citizens of the world?', 1.2047890424728394)


In [ ]:
query = ['Inductive reasoning leads to the development of what type of thinking?']
top_k_ques(query)

	 ('What did Burke think a social hierarchy should be based on?', 1.1677690744400024)
	 ('What philosophy did Price support?', 1.1684865951538086)
	 ('What type of doctrine did Burke propose?', 1.1690744161605835)
	 ('What did Burke believe was against the natural order of events?', 1.1744353771209717)
	 ('Whose principles did Burke imitate?', 1.1752909421920776)


In [ ]:
query = ['Inductive reasoning leads to the development of what type of thinking?']
top_k_ques(query)

	 ('What did Burke think a social hierarchy should be based on?', 1.1677690744400024)
	 ('What philosophy did Price support?', 1.1684865951538086)
	 ('What type of doctrine did Burke propose?', 1.1690744161605835)
	 ('What did Burke believe was against the natural order of events?', 1.1744353771209717)
	 ('Whose principles did Burke imitate?', 1.1752909421920776)


In [ ]:
query = ['Inductive reasoning leads to the development of what type of thinking?']
top_k_ques(query)

	 ('What did Burke think a social hierarchy should be based on?', 1.1677690744400024)
	 ('What philosophy did Price support?', 1.1684865951538086)
	 ('What type of doctrine did Burke propose?', 1.1690744161605835)
	 ('What did Burke believe was against the natural order of events?', 1.1744353771209717)
	 ('Whose principles did Burke imitate?', 1.1752909421920776)


In [ ]:
query = ['Inductive reasoning leads to the development of what type of thinking?']
top_k_ques(query)

	 ('What did Burke think a social hierarchy should be based on?', 1.1677690744400024)
	 ('What philosophy did Price support?', 1.1684865951538086)
	 ('What type of doctrine did Burke propose?', 1.1690744161605835)
	 ('What did Burke believe was against the natural order of events?', 1.1744353771209717)
	 ('Whose principles did Burke imitate?', 1.1752909421920776)


#Theme 11 - National_Archives_and_Records_Administration 

In [ ]:
theme = 'National_Archives_and_Records_Administration'

##Analysis

Similar

In [ ]:
query = ['What collection of archives is under the jurisdiction of the National Archives and Records Administration (NARA)?']
top_k_ques(query)

	 ('The Annual Register covered international events of what type?', 1.1278140544891357)
	 ('What pamphlet did Fox publish in 3 August 1791?', 1.1755728721618652)
	 ('What was Hastings the governor of during his impeachment?', 1.186761736869812)
	 ('What was the impeachment formally called?', 1.1943436861038208)
	 ('Who was the Secretary of the Board of Agriculture?', 1.1950229406356812)


In [ ]:
query = ['Which independent organization preserves the original copy of executive orders?']
top_k_ques(query)

	 ('Which administration proposed the Paymaster act?', 1.0631524324417114)
	 ('Who wanted to limit the powers of the political parties?', 1.144372820854187)
	 ('What pamphlet did Fox publish in 3 August 1791?', 1.153132438659668)
	 ('What did Burke call corporate tyranny in India?', 1.159178614616394)
	 ('Who was the Secretary of the Board of Agriculture?', 1.1655924320220947)


In [ ]:
query = ['What is the responsibility of Congress in terms of preserving historical documents?']
top_k_ques(query)

	 ('Who wanted to limit the powers of the political parties?', 1.1469264030456543)
	 ('What did the Whig party want Burke to condemn?', 1.1509652137756348)
	 ('What did Burke believe made political arrangements difficult?', 1.1614875793457031)
	 ('When was the American Declaration of Independence issued?', 1.1667201519012451)
	 ('Which administration proposed the Paymaster act?', 1.178993582725525)


In [ ]:
query = ['What type of entity is The Electoral College?']
top_k_ques(query)

	 ('What was the impeachment formally called?', 1.1239614486694336)
	 ('Who wanted to limit the powers of the political parties?', 1.1316622495651245)
	 ('What political party did Johnson claim the angel represented?', 1.1672425270080566)
	 ('Who did Burke want constitutional limits on the power of?', 1.1717947721481323)
	 ('What type of government did Burke think would be inept?', 1.1812896728515625)


In [ ]:
query = ['What kinds of records does Congress maintain?']
top_k_ques(query)

	 ('What was published in 1775?', 1.1248985528945923)
	 ('What pamphlet did Fox publish in 3 August 1791?', 1.161558985710144)
	 ('The Annual Register covered international events of what type?', 1.1761012077331543)
	 ('What was the impeachment formally called?', 1.1802570819854736)
	 ('What was Hastings the governor of during his impeachment?', 1.1808199882507324)


In [ ]:
query = ['What type of entity is The Electoral College?']
top_k_ques(query)

	 ('What was the impeachment formally called?', 1.123961329460144)
	 ('Who wanted to limit the powers of the political parties?', 1.1316622495651245)
	 ('What political party did Johnson claim the angel represented?', 1.1672425270080566)
	 ('Who did Burke want constitutional limits on the power of?', 1.1717948913574219)
	 ('What type of government did Burke think would be inept?', 1.181289553642273)


In [ ]:
query = ['What kinds of records does Congress maintain?']
top_k_ques(query)

	 ('What was published in 1775?', 1.1248985528945923)
	 ('What pamphlet did Fox publish in 3 August 1791?', 1.161558985710144)
	 ('The Annual Register covered international events of what type?', 1.1761012077331543)
	 ('What was the impeachment formally called?', 1.1802570819854736)
	 ('What was Hastings the governor of during his impeachment?', 1.1808199882507324)


In [ ]:
query = ['What areas of governance do state legislatures have authority over?']
top_k_ques(query)

	 ('Who wanted to limit the powers of the political parties?', 1.0743677616119385)
	 ('What type of government did Burke think would be inept?', 1.1201250553131104)
	 ('Who did Burke want constitutional limits on the power of?', 1.1225531101226807)
	 ('What did Burke say the plan of freedom must be governed by?', 1.1345014572143555)
	 ('What did Burke think the British Empire should be governed based on?', 1.137668251991272)


In [ ]:
query = ['What type of records are kept by state legislatures?']
top_k_ques(query)

	 ('What was published in 1775?', 1.1257001161575317)
	 ('What was Hastings the governor of during his impeachment?', 1.1590280532836914)
	 ('What pamphlet did Fox publish in 3 August 1791?', 1.1695959568023682)
	 ('What pamphlet was published on 13 February 1790?', 1.1758121252059937)
	 ('The Annual Register covered international events of what type?', 1.1776270866394043)


In [ ]:
query = ['What powers do state legislatures hold?']
top_k_ques(query)

	 ('Who wanted to limit the powers of the political parties?', 1.0824716091156006)
	 ('Who did Burke want constitutional limits on the power of?', 1.1050313711166382)
	 ('What did Burke think the British Empire should be governed based on?', 1.1280463933944702)
	 ('Which constitution did Burke condemn on May 6, 1791?', 1.1285582780838013)
	 ('What did Burke believe made political arrangements difficult?', 1.1330187320709229)


In [ ]:
query = ['Which office manages the Electoral College?']
top_k_ques(query)

	 ('What was the impeachment formally called?', 1.1129554510116577)
	 ('Which administration proposed the Paymaster act?', 1.1361970901489258)
	 ('Who wanted to limit the powers of the political parties?', 1.1416020393371582)
	 ('Who condemned the Canadian constitution?', 1.1504417657852173)
	 ('Who did Burke want constitutional limits on the power of?', 1.157193660736084)


In [ ]:
query = ['What is the Electoral College responsible for implementing?']
top_k_ques(query)

	 ('Who wanted to limit the powers of the political parties?', 1.081020712852478)
	 ('What was the impeachment formally called?', 1.0972782373428345)
	 ('Who did Burke want constitutional limits on the power of?', 1.1226732730865479)
	 ('Who condemned the Canadian constitution?', 1.1305218935012817)
	 ('Which administration proposed the Paymaster act?', 1.1595189571380615)


In [ ]:
query = ['What regulations are issued by the Electoral College?']
top_k_ques(query)

	 ('What was the impeachment formally called?', 1.1234490871429443)
	 ('What law was William Ewart Gladstone against?', 1.1415411233901978)
	 ('Which administration proposed the Paymaster act?', 1.145695447921753)
	 ('Who wanted to limit the powers of the political parties?', 1.160748839378357)
	 ('What was published in 1775?', 1.1768014430999756)


In [ ]:
query = ['What is the main function of the office administered by the Electoral College?']
top_k_ques(query)

	 ('What was the impeachment formally called?', 1.1253501176834106)
	 ('Who wanted to limit the powers of the political parties?', 1.1652545928955078)
	 ('What was Hastings the governor of during his impeachment?', 1.175482988357544)
	 ('What did Burke say the plan of freedom must be governed by?', 1.1799489259719849)
	 ('Which administration proposed the Paymaster act?', 1.1888591051101685)


In [ ]:
query = ['In what year was the National Historical Publications and Records Commission established?']
top_k_ques(query)

	 ("When was Burke's history of England published?", 0.9441933035850525)
	 ("When was Hume's history of England published?", 0.9916211366653442)
	 ('When was the American Declaration of Independence issued?', 1.0090419054031372)
	 ('What was founded the year before the contract?', 1.018351674079895)
	 ('Dodsley was the only writer for the register until what year?', 1.0227359533309937)


In [ ]:
query = ['How many grants are offered by the government each year?']
top_k_ques(query)

	 ('When did Paymasters begin to get money directly from the HM treasury?', 1.1630337238311768)
	 ('How many books did Lord Acton publish on Burke?', 1.1896922588348389)
	 ('How much money did Burke accept instead of the Earlship?', 1.1939005851745605)
	 ('Which administration proposed the Paymaster act?', 1.194284439086914)
	 ('How much did most pamphlets cost at the time?', 1.2039951086044312)


In [ ]:
query = ['How many institutions of higher education award grants to students annually?']
top_k_ques(query)

	 ('In what year did Trinity College ban Catholics?', 1.1751765012741089)
	 ('When was Trinity College Dublin established?', 1.1820307970046997)
	 ('When did Paymasters begin to get money directly from the HM treasury?', 1.1865133047103882)
	 ('How many books did Lord Acton publish on Burke?', 1.18731689453125)
	 ('When did Burke begin attending Trinity College Dublin?', 1.197001338005066)


Dissimilar

In [ ]:
query = ['NARA is not responsible for what collection of archives??']
top_k_ques(query)

	 ('Who wanted the argument to go public?', 1.1090234518051147)
	 ('Dodsley was the only writer for the register until what year?', 1.126320719718933)
	 ("Who said he 'perfectly differed from Mr. Fox'?", 1.1327275037765503)
	 ('The Annual Register covered international events of what type?', 1.1409986019134521)
	 ("Fox quoted Burke's speeches from how long ago?", 1.1558775901794434)


In [ ]:
query = ['What independent agency does not preserve the original copy of executive orders?']
top_k_ques(query)

	 ('Which administration proposed the Paymaster act?', 1.0634586811065674)
	 ('Who was the Secretary of the Board of Agriculture?', 1.1621464490890503)
	 ('What pamphlet did Fox publish in 3 August 1791?', 1.1663269996643066)
	 ('Who could no longer make withdrawal requests to the Bank of England after 1782?', 1.1790878772735596)
	 ('Who did Basset send a copy of his letter to?', 1.1804219484329224)


In [ ]:
query = ['What is Congress not responsible for preserving?']
top_k_ques(query)

	 ('Who wanted to limit the powers of the political parties?', 1.1250113248825073)
	 ('What did the Whig party want Burke to condemn?', 1.1505616903305054)
	 ('Who did Burke want constitutional limits on the power of?', 1.1568589210510254)
	 ('What was the impeachment formally called?', 1.1633062362670898)
	 ('Which administration proposed the Paymaster act?', 1.1673842668533325)


In [ ]:
query = ['What kind of agency is not The Electoral College?']
top_k_ques(query)

	 ('What was the impeachment formally called?', 1.1544065475463867)
	 ('Which administration proposed the Paymaster act?', 1.1724590063095093)
	 ('What type of government did Burke think would be inept?', 1.1920720338821411)
	 ('Who wanted to limit the powers of the political parties?', 1.2021667957305908)
	 ('Who was the Secretary of the Board of Agriculture?', 1.2058130502700806)


In [ ]:
query = ['What types of documents does not Congress preserve?']
top_k_ques(query)

	 ('What was published in 1775?', 1.1260175704956055)
	 ('What pamphlet did Fox publish in 3 August 1791?', 1.1363000869750977)
	 ('Which constitution did Burke condemn on May 6, 1791?', 1.1492518186569214)
	 ('What was the impeachment formally called?', 1.161470890045166)
	 ('What pamphlet was published on 13 February 1790?', 1.1672848463058472)


In [ ]:
query = ['What does state legislatures not oversee the operation of?']
top_k_ques(query)

	 ('Who wanted to limit the powers of the political parties?', 1.0323536396026611)
	 ('Who did Burke want constitutional limits on the power of?', 1.070587396621704)
	 ('What did Burke say the plan of freedom must be governed by?', 1.1062129735946655)
	 ('What did Burke believe made political arrangements difficult?', 1.1116529703140259)
	 ('What did the Whig party want Burke to condemn?', 1.1213151216506958)


In [ ]:
query = ['What kind of documentation is not maintained by the state legislatures?']
top_k_ques(query)

	 ('What was published in 1775?', 1.0966675281524658)
	 ('What did Burke believe made political arrangements difficult?', 1.1220911741256714)
	 ('What pamphlet did Fox publish in 3 August 1791?', 1.1269738674163818)
	 ('What pamphlet was published on 13 February 1790?', 1.128035545349121)
	 ('Which constitution did Burke condemn on May 6, 1791?', 1.1280832290649414)


In [ ]:
query = ['What authority does state legislatures not have?']
top_k_ques(query)

	 ('Who wanted to limit the powers of the political parties?', 1.0536378622055054)
	 ('Who did Burke want constitutional limits on the power of?', 1.0659152269363403)
	 ('Which constitution did Burke condemn on May 6, 1791?', 1.079596757888794)
	 ('What type of government did Burke think would be inept?', 1.0969607830047607)
	 ('What did Burke think the British Empire should be governed based on?', 1.09807550907135)


In [ ]:
query = ['What office does not administer the Electoral College?']
top_k_ques(query)

	 ('What was the impeachment formally called?', 1.095910906791687)
	 ('Which administration proposed the Paymaster act?', 1.126145601272583)
	 ('Which constitution did Burke condemn on May 6, 1791?', 1.152563452720642)
	 ('Who wanted to limit the powers of the political parties?', 1.1629735231399536)
	 ('Who did Burke want constitutional limits on the power of?', 1.1682780981063843)


In [ ]:
query = ['What code is the Electoral College not responsible for?']
top_k_ques(query)

	 ('What was the impeachment formally called?', 1.052178144454956)
	 ('Who condemned the Canadian constitution?', 1.1530561447143555)
	 ('When was the American Declaration of Independence issued?', 1.1714086532592773)
	 ('Which constitution did Burke condemn on May 6, 1791?', 1.1850873231887817)
	 ('What was Hastings the governor of during his impeachment?', 1.1885653734207153)


In [ ]:
query = ['What statutes does the Electoral College not publish?']
top_k_ques(query)

	 ('What was published in 1775?', 1.0718313455581665)
	 ('Which constitution did Burke condemn on May 6, 1791?', 1.075891137123108)
	 ('What was the impeachment formally called?', 1.086777925491333)
	 ('What pamphlet did Fox publish in 3 August 1791?', 1.0868772268295288)
	 ('What law was William Ewart Gladstone against?', 1.0905219316482544)


In [ ]:
query = ['What office does the Electoral College not administer?']
top_k_ques(query)

	 ('What was the impeachment formally called?', 1.0893559455871582)
	 ('Which administration proposed the Paymaster act?', 1.1576879024505615)
	 ('Which constitution did Burke condemn on May 6, 1791?', 1.1676931381225586)
	 ('Who did Burke want constitutional limits on the power of?', 1.1932709217071533)
	 ('Who condemned the Canadian constitution?', 1.193400263786316)


In [ ]:
query = ['What year was The National Historical Publications and Records Commission not established?']
top_k_ques(query)

	 ("When was Burke's history of England published?", 0.935650110244751)
	 ("When was Hume's history of England published?", 0.9856116771697998)
	 ('When was the American Declaration of Independence issued?', 1.0012439489364624)
	 ('Dodsley was the only writer for the register until what year?', 1.0056188106536865)
	 ('What was founded the year before the contract?', 1.0152335166931152)


In [ ]:
query = ['About how many grants has the NHPRC not awarded?']
top_k_ques(query)

	 ('When did Paymasters begin to get money directly from the HM treasury?', 1.123218297958374)
	 ('How much money did Burke accept instead of the Earlship?', 1.1616883277893066)
	 ('How many books did Lord Acton publish on Burke?', 1.1668037176132202)
	 ("How many words long was Burke's history of England contracted to be?", 1.2025532722473145)
	 ("How many pages long was Burke's history of England contracted to be?", 1.2054669857025146)


In [ ]:
query = ['How many grants are not awarded by the government annually?']
top_k_ques(query)

	 ('When did Paymasters begin to get money directly from the HM treasury?', 1.1643184423446655)
	 ('How many books did Lord Acton publish on Burke?', 1.1996713876724243)
	 ('Which administration proposed the Paymaster act?', 1.214601755142212)
	 ('How much money did Burke accept instead of the Earlship?', 1.2234889268875122)
	 ('What was founded the year before the contract?', 1.227081537246704)


In [ ]:
query = ['How many colleges and universities not award grants to students each year?']
top_k_ques(query)

	 ('In what year did Trinity College ban Catholics?', 1.1771771907806396)
	 ('What type of people did Trinity College Dublin not allow to earn degrees there?', 1.203924298286438)
	 ('When did Paymasters begin to get money directly from the HM treasury?', 1.2124168872833252)
	 ('How many books did Lord Acton publish on Burke?', 1.2124898433685303)
	 ('When was Trinity College Dublin established?', 1.2199974060058594)


#Theme 12 - Hindu_philosophy

In [ ]:
theme = 'Hindu_philosophy'

##Analysis

Similar

In [ ]:
query = ['What is the historical origin of Hindu philosophy?']
top_k_ques(query)

	 ('What philosophy did Price support?', 1.1242917776107788)
	 ('What did Burke believe was the foundation of religion?', 1.1352299451828003)
	 ('Whose principles did L. Bol imitate?', 1.150197982788086)
	 ('When was the Revolution Society founded?', 1.157507300376892)
	 ("Who wrote 'The Rights of Man'?", 1.1651804447174072)


In [ ]:
query = ['How many main branches of Hindu philosophy exist?']
top_k_ques(query)

	 ('What did Burke call corporate tyranny in India?', 1.1911574602127075)
	 ('How many books did Lord Acton publish on Burke?', 1.2175661325454712)
	 ('What philosophy did Price support?', 1.2183412313461304)
	 ('What did Burke believe was the foundation of religion?', 1.2218865156173706)
	 ('Whose principles did L. Bol imitate?', 1.223077416419983)


In [ ]:
query = ['What are the traditional, mainstream systems of Hindu philosophy referred to as?']
top_k_ques(query)

	 ('What did Burke call corporate tyranny in India?', 1.0492324829101562)
	 ('What philosophy did Price support?', 1.1120572090148926)
	 ('What type of doctrine did Burke propose?', 1.144097089767456)
	 ('Whose principles did L. Bol imitate?', 1.1459842920303345)
	 ('What did Paine call Reflections?', 1.176249623298645)


In [ ]:
query = ['What are the non-traditional Indian philosophical systems that do not accept the authority of the Vedas called?']
top_k_ques(query)

	 ('What did Burke call corporate tyranny in India?', 1.013529896736145)
	 ('What did Burke nickname his paternalistic doctrine?', 1.1663790941238403)
	 ('What philosophy did Price support?', 1.1758759021759033)
	 ('What type of doctrine did Burke propose?', 1.1890021562576294)
	 ('What did Paine call Reflections?', 1.1898236274719238)


In [ ]:
query = ['What is the meaning of the term "Carvaka"?']
top_k_ques(query)

	 ("Who said he 'perfectly differed from Mr. Fox'?", 1.165772557258606)
	 ('What did Charles James Fox translate?', 1.1755249500274658)
	 ('Who wanted the argument to go public?', 1.1875784397125244)
	 ("What did Burke say 'envenoms everything it touches'?", 1.1945948600769043)
	 ('Whose ascendancy did Burke think was malignant?', 1.2203004360198975)


In [ ]:
query = ['In what centuries have scholars used primary texts to debate Indian philosophical traditions?']
top_k_ques(query)

	 ("When was Hume's history of England published?", 1.0724375247955322)
	 ("When was the tenth printing of the French translation of 'Reflections on the Revolution in France'?", 1.093199372291565)
	 ('When did Edmund Burke start a debate club?', 1.101587176322937)
	 ("How long before Hastings's trial had Britain been dealing with Indian unrest?", 1.1114702224731445)
	 ('In what year did Trinity College ban Catholics?', 1.113917350769043)


In [ ]:
query = ['How are Buddhism and Jainism classified in relation to religion and philosophy?']
top_k_ques(query)

	 ('What philosophy did Price support?', 1.229480266571045)
	 ('What did Burke believe was the foundation of religion?', 1.2309048175811768)
	 ('What did Burke say established religion undermined?', 1.2533445358276367)
	 ('Burke thought religion was beneficial to what besides souls?', 1.260706901550293)
	 ('Who converted into Catholicism?', 1.264814019203186)


In [ ]:
query = ['What other traditions are considered to be within the realm of Hindu philosophy?']
top_k_ques(query)

	 ('Whose principles did L. Bol imitate?', 1.1656644344329834)
	 ('What philosophy did Price support?', 1.1719157695770264)
	 ('Burke thought religion was beneficial to what besides souls?', 1.2155579328536987)
	 ('What did Burke call corporate tyranny in India?', 1.2161716222763062)
	 ('What did Burke believe was the foundation of religion?', 1.2186309099197388)


In [ ]:
query = ['How accessible are the primary sources of knowledge on Indian philosophy?']
top_k_ques(query)

	 ('What vice-filled book did Hume publish?', 1.1971913576126099)
	 ('What did Rousseau say Burke had in regards to understanding human nature?', 1.1989880800247192)
	 ("When was Hume's history of England published?", 1.2055028676986694)
	 ('What philosophy did Price support?', 1.210242748260498)
	 ('What publication did Burke found?', 1.215851068496704)


In [ ]:
query = ['What are the key differences between the various sibling traditions within Indian philosophy?']
top_k_ques(query)

	 ('What did Rousseau say Burke had in regards to understanding human nature?', 1.1929458379745483)
	 ('Whose principles did L. Bol imitate?', 1.195361852645874)
	 ('What philosophy did Price support?', 1.2086570262908936)
	 ('What did Burke think a social hierarchy should be based on?', 1.2359619140625)
	 ('What did Burke think had disgraced Britain in India?', 1.244188904762268)


In [ ]:
query = ['What points of agreement exist within the Astika and Nastika philosophical systems?']
top_k_ques(query)

	 ('What did Rousseau say Burke had in regards to understanding human nature?', 1.1017093658447266)
	 ('What philosophy did Price support?', 1.1136760711669922)
	 ('What did Burke think a social hierarchy should be based on?', 1.1336159706115723)
	 ('What type of doctrine did Burke propose?', 1.1389844417572021)
	 ("What did Lord Chesterfield believe Bolingbroke's arguments could also apply to?", 1.1404283046722412)


In [ ]:
query = ['Which two named philosophical systems are considered to be identical?']
top_k_ques(query)

	 ('What philosophy did Price support?', 1.1051099300384521)
	 ('What type of doctrine did Burke propose?', 1.1466470956802368)
	 ('Whose principles did Burke imitate?', 1.147693157196045)
	 ('What did Rousseau say Burke had in regards to understanding human nature?', 1.1576658487319946)
	 ('Which French philosopher did Burke attack?', 1.1591163873672485)


In [ ]:
query = ['How many traditionally recognized systems of philosophy exist within Hinduism?']
top_k_ques(query)

	 ('What philosophy did Price support?', 1.156436800956726)
	 ('What did Burke call corporate tyranny in India?', 1.1957658529281616)
	 ('Who converted into Catholicism?', 1.206263542175293)
	 ('When was Trinity College Dublin established?', 1.2090113162994385)
	 ('Which French philosopher did Burke attack?', 1.2162872552871704)


Disimilar

In [ ]:
query = ['What was not the origin of Hindu philosophy?']
top_k_ques(query)

	 ('What philosophy did Price support?', 1.1098543405532837)
	 ('What did Burke believe was the foundation of religion?', 1.1200270652770996)
	 ('Whose principles did L. Bol imitate?', 1.131596565246582)
	 ('What did Burke call corporate tyranny in India?', 1.161873698234558)
	 ("Who wrote 'The Rights of Man'?", 1.1630444526672363)


In [ ]:
query = ['How many main systems of Hindu philosophy are not there?']
top_k_ques(query)

	 ('What did Burke call corporate tyranny in India?', 1.1768474578857422)
	 ('What philosophy did Price support?', 1.1982718706130981)
	 ('How many books did Lord Acton publish on Burke?', 1.2093274593353271)
	 ('What did Burke believe was the foundation of religion?', 1.2293974161148071)
	 ('What vice-filled book did Hume publish?', 1.229662299156189)


In [ ]:
query = ['What are not the main orthodox systems of Hindu philosophy called?']
top_k_ques(query)

	 ('What did Burke call corporate tyranny in India?', 1.0597339868545532)
	 ('What philosophy did Price support?', 1.1456793546676636)
	 ('What did Burke nickname his paternalistic doctrine?', 1.168890118598938)
	 ('What type of doctrine did Burke propose?', 1.1690785884857178)
	 ('Whose principles did L. Bol imitate?', 1.1864662170410156)


In [ ]:
query = ['What is not the accepted source of knowledge in the Astika system?']
top_k_ques(query)

	 ('What did Rousseau say Burke had in regards to understanding human nature?', 1.1756504774093628)
	 ('What did Burke believe was against the natural order of events?', 1.1814168691635132)
	 ('What did Burke believe was the foundation of religion?', 1.1819347143173218)
	 ('What publication did Burke found?', 1.18936026096344)
	 ('What did Burke think was the foundation of society?', 1.1987594366073608)


In [ ]:
query = ['How many darsanas are not there?']
top_k_ques(query)

	 ('Who wanted the argument to go public?', 0.815747857093811)
	 ("What did Burke say 'envenoms everything it touches'?", 1.0077152252197266)
	 ("Who said he 'perfectly differed from Mr. Fox'?", 1.0310702323913574)
	 ("Fox quoted Burke's speeches from how long ago?", 1.1442654132843018)
	 ('Fox quoted his own speeches from how long ago?', 1.1492787599563599)


In [ ]:
query = ['What does Carvaka not mean?']
top_k_ques(query)

	 ("Who said he 'perfectly differed from Mr. Fox'?", 1.1553152799606323)
	 ("What did Burke say 'envenoms everything it touches'?", 1.1712241172790527)
	 ('What did Charles James Fox translate?', 1.172196388244629)
	 ('Who wanted the argument to go public?', 1.175680160522461)
	 ('Who claimed to be similar to Mr. Fox?', 1.2473191022872925)


In [ ]:
query = ['In what centuries the writings of scholars not been used to debate the philosophies?']
top_k_ques(query)

	 ("When was Hume's history of England published?", 1.0950448513031006)
	 ('When did Edmund Burke start a debate club?', 1.0954574346542358)
	 ("When did Marie-Antoinette read Burke's writings?", 1.1011003255844116)
	 ("When was the tenth printing of the French translation of 'Reflections on the Revolution in France'?", 1.1090126037597656)
	 ('When did Burke publish his Reflections on the Revolution in France?', 1.1155263185501099)


In [ ]:
query = ['What are Buddhism and Jainism not regarded as in relation to religion and philosophy?']
top_k_ques(query)

	 ('What philosophy did Price support?', 1.1927310228347778)
	 ('What did Burke believe was the foundation of religion?', 1.2030457258224487)
	 ('Burke thought religion was beneficial to what besides souls?', 1.2296305894851685)
	 ('What did Burke call corporate tyranny in India?', 1.231122612953186)
	 ('What did Christopher Hitchens believe was essential to human life?', 1.2388882637023926)


In [ ]:
query = ['What are not other schools regarded as within Hindu philosophy?']
top_k_ques(query)

	 ('What type of people did Trinity College Dublin not allow to earn degrees there?', 1.1779128313064575)
	 ('What did Burke call corporate tyranny in India?', 1.1811903715133667)
	 ('What philosophy did Price support?', 1.2016308307647705)
	 ('Whose principles did L. Bol imitate?', 1.205251932144165)
	 ('Burke thought religion was beneficial to what besides souls?', 1.2162137031555176)


In [ ]:
query = ['How not available are the sources of knowledge on Indian philosophy?']
top_k_ques(query)

	 ('What did Rousseau say Burke had in regards to understanding human nature?', 1.1964689493179321)
	 ('What vice-filled book did Hume publish?', 1.2149263620376587)
	 ("When was Hume's history of England published?", 1.2246191501617432)
	 ("Why did a few scholars think Burke hadn't intended his book as a satire yet later claimed it was a satire?", 1.2448720932006836)
	 ("How long before Hastings's trial had Britain been dealing with Indian unrest?", 1.2473376989364624)


In [ ]:
query = ['What does not differentiate the various sibling traditions?']
top_k_ques(query)

	 ('Whose principles did L. Bol imitate?', 1.199721336364746)
	 ('Whose principles did Burke imitate?', 1.213470220565796)
	 ('What did Burke think a social hierarchy should be based on?', 1.215052604675293)
	 ('What did Burke believe was against the natural order of events?', 1.2285244464874268)
	 ('How did Burke say we look towards nobility?', 1.249155044555664)


In [ ]:
query = ['What do scholars not agree on within the astika and nastika philosophies?']
top_k_ques(query)

	 ("What did Lord Chesterfield believe Bolingbroke's arguments could also apply to?", 1.0995430946350098)
	 ('What did Rousseau say Burke had in regards to understanding human nature?', 1.1206274032592773)
	 ('What did Burke believe was against the natural order of events?', 1.1477584838867188)
	 ('What did Burke believe was the foundation of religion?', 1.1555795669555664)
	 ('What did the Whig party want Burke to condemn?', 1.160893440246582)


In [ ]:
query = ['Which two named philosophies are not considered to be the same?']
top_k_ques(query)

	 ('What philosophy did Price support?', 1.1078741550445557)
	 ('What type of doctrine did Burke propose?', 1.122483730316162)
	 ('Whose principles did Burke imitate?', 1.124170184135437)
	 ('Who did Thomas Paine compare Burke to?', 1.139926791191101)
	 ('Burke thought the French Revolution was against whose principles?', 1.1454613208770752)


In [ ]:
query = ['How many not orthodox philosophies are in Hindu?']
top_k_ques(query)

	 ('How many books did Lord Acton publish on Burke?', 1.2027277946472168)
	 ("How many words was Lord Acton's commentary?", 1.2058309316635132)
	 ('Who converted into Catholicism?', 1.2104698419570923)
	 ('What did Burke call corporate tyranny in India?', 1.2144889831542969)
	 ('Who practiced Protestant ascendency?', 1.2204737663269043)


#Theme 13 - YouTube

In [ ]:
theme = 'YouTube'

##Analysis

Similar

In [ ]:
query = ['What was the date when Youtube started of?']
top_k_ques(query)

	 ('What was founded the year before the contract?', 0.9539806246757507)
	 ('When was the Revolution Society founded?', 1.0081782341003418)
	 ('When was the American Declaration of Independence issued?', 1.0411734580993652)
	 ('When was Trinity College Dublin established?', 1.0530281066894531)
	 ('Dodsley was the only writer for the register until what year?', 1.0714589357376099)


In [ ]:
query = ['When did the idea of Youtube emerge in the mind of the creators?']
top_k_ques(query)

	 ('What was founded the year before the contract?', 1.0729528665542603)
	 ('When was the Revolution Society founded?', 1.0799319744110107)
	 ('When did Paymasters begin to get money directly from the HM treasury?', 1.1095572710037231)
	 ('When was Trinity College Dublin established?', 1.1261759996414185)
	 ('When did Burke become the chief editor?', 1.1278623342514038)


In [ ]:
query = ['In 2006, how much did Google pay for Youtube?']
top_k_ques(query)

	 ('How much did most pamphlets cost at the time?', 1.074819564819336)
	 ('How much money did Burke accept instead of the Earlship?', 1.0951895713806152)
	 ('What was founded the year before the contract?', 1.1075825691223145)
	 ('When did Paymasters begin to get money directly from the HM treasury?', 1.1895380020141602)
	 ('Where had Paymasters been able to get money from directly until 1782?', 1.204347848892212)


In [ ]:
query = ['What other content is on YouTube, besides video blogging and instructional videos?']
top_k_ques(query)

	 ('What was published in 1775?', 1.2377465963363647)
	 ('What did François-Louis-Thibault de Menonville publish in April 1791?', 1.2530878782272339)
	 ('What pamphlet did Fox publish in 3 August 1791?', 1.2542049884796143)
	 ("What was Pitt's last publication?", 1.26946222782135)
	 ('What publication did Burke found?', 1.274274468421936)


In [ ]:
query = ['Who gave US$1.65 billion for YouTube in 2005?']
top_k_ques(query)

	 ('What was founded the year before the contract?', 1.1633059978485107)
	 ('Who could no longer make withdrawal requests to the Bank of England after 1782?', 1.1803767681121826)
	 ('Where had Paymasters been able to get money from directly until 1782?', 1.1818119287490845)
	 ('When did Paymasters begin to get money directly from the HM treasury?', 1.187896490097046)
	 ("Who wrote 'The Rights of Man'?", 1.192306637763977)


In [ ]:
query = ['Who invented youtube, mention their first names']
top_k_ques(query)

	 ("Who wrote 'The Rights of Man'?", 1.0712089538574219)
	 ("Who wrote Dodsley's biography?", 1.1002025604248047)
	 ('Who introduced a bill in 1795 about minimum wage?', 1.1314343214035034)
	 ('Dodsley was the only writer for the register until what year?', 1.1478828191757202)
	 ('Who wrote a biography of Burke?', 1.1505300998687744)


In [ ]:
query = ['What was the resaon behind website?']
top_k_ques(query)

	 ('Who wanted the argument to go public?', 1.0493507385253906)
	 ("What did Burke say 'envenoms everything it touches'?", 1.0772327184677124)
	 ('What did Charles James Fox translate?', 1.1461759805679321)
	 ('What was founded the year before the contract?', 1.1580289602279663)
	 ('Where had Paymasters been able to get money from directly until 1782?', 1.1626241207122803)


In [ ]:
query = ['When did the idea of Google come up in the minds of creators? ']
top_k_ques(query)

	 ('When was the Revolution Society founded?', 1.0669958591461182)
	 ('What was founded the year before the contract?', 1.0689945220947266)
	 ('When was Trinity College Dublin established?', 1.1091278791427612)
	 ('When did Burke become the chief editor?', 1.1174492835998535)
	 ('When did Paymasters begin to get money directly from the HM treasury?', 1.12501859664917)


In [ ]:
query = ['What was not the initial motivation for developing the website ?']
top_k_ques(query)

	 ('What was founded the year before the contract?', 1.139086365699768)
	 ('What did Hitchens appeal to?', 1.196720004081726)
	 ('What philosophy did Price support?', 1.1971840858459473)
	 ("What did critics particularly protest of Burke's work?", 1.198240876197815)
	 ('Where had Paymasters been able to get money from directly until 1782?', 1.2096439599990845)


In [ ]:
query = ['Who was involved in selling youtube?']
top_k_ques(query)

	 ('Where had Paymasters been able to get money from directly until 1782?', 1.112015962600708)
	 ("Who wrote 'The Rights of Man'?", 1.1593741178512573)
	 ('What was founded the year before the contract?', 1.1621078252792358)
	 ('Who converted into Catholicism?', 1.1739400625228882)
	 ('Who practiced Protestant ascendency?', 1.1769808530807495)


In [ ]:
query = ['When was the idea for youtube developed?']
top_k_ques(query)

	 ('What was founded the year before the contract?', 1.010378122329712)
	 ('When was the Revolution Society founded?', 1.0367299318313599)
	 ('When was Trinity College Dublin established?', 1.0691953897476196)
	 ('When was trade with Ireland originally restricted?', 1.100154161453247)
	 ('Dodsley was the only writer for the register until what year?', 1.1320488452911377)


In [ ]:
query = ['When did the first youtube beta testing take place>']
top_k_ques(query)

	 ("How long before Hastings's trial had Britain been dealing with Indian unrest?", 1.1226402521133423)
	 ('What was founded the year before the contract?', 1.136035442352295)
	 ('When did Paymasters begin to get money directly from the HM treasury?', 1.1420449018478394)
	 ('Dodsley was the only writer for the register until what year?', 1.1436516046524048)
	 ('When did Wordsworth initially attack Burke?', 1.1493841409683228)


In [ ]:
query = ['How much was invested by Sequoia Capital invest in youtube in November of 2005?']
top_k_ques(query)

	 ('How much money did Burke accept instead of the Earlship?', 1.1707218885421753)
	 ('What was founded the year before the contract?', 1.170929193496704)
	 ('How much did most pamphlets cost at the time?', 1.1727559566497803)
	 ("How many words was Lord Acton's commentary?", 1.198289155960083)
	 ('How long was the book Robert Dodsley wrote?', 1.20188307762146)


In [ ]:
query = ['By the end of May 2010 how many views did Youtube achive?']
top_k_ques(query)

	 ('How much did most pamphlets cost at the time?', 1.186821460723877)
	 ('What pamphlet did Fox publish in 3 August 1791?', 1.2039427757263184)
	 ('How many books did Lord Acton publish on Burke?', 1.2063720226287842)
	 ("How many words was Lord Acton's commentary?", 1.2148282527923584)
	 ('What was founded the year before the contract?', 1.2267709970474243)


In [ ]:
query = ['Which event took place in March 2005?']
top_k_ques(query)

	 ('What was the Revolution Society commemorating?', 1.142948031425476)
	 ('When did Burke make a motion to restrict Irish trade?', 1.1803275346755981)
	 ('Which victories against the Americans did the British celebrate?', 1.1810097694396973)
	 ('What was founded the year before the contract?', 1.185823678970337)
	 ('When did Mary meet Burke?', 1.1866611242294312)


In [ ]:
query = ['Which video became youtube second video to reach a million views?']
top_k_ques(query)

	 ('What pamphlet did Fox publish in 3 August 1791?', 1.2071688175201416)
	 ('What pamphlet was published on 13 February 1790?', 1.2326191663742065)
	 ("What was Pitt's last publication?", 1.2434806823730469)
	 ('What was published in 1775?', 1.2478512525558472)
	 ('What did François-Louis-Thibault de Menonville publish in April 1791?', 1.2510991096496582)


In [ ]:
query = ['On May 2006 what public test was offered ?']
top_k_ques(query)

	 ('What pamphlet was published on 13 February 1790?', 1.100520372390747)
	 ('What pamphlet did Fox publish in 3 August 1791?', 1.1005884408950806)
	 ('What was published in 1775?', 1.1288021802902222)
	 ("What was Burke's final publication?", 1.132952094078064)
	 ('Which administration proposed the Paymaster act?', 1.1347852945327759)


For annoy and universal encoder:- \
When senseless negation of questions are compared, similarity score is seen as low as 0.18739761412143707 \
  For similar questions lowest threshold below which it is always right = 0.2109205573797226\
  For similar questions highest threshold for first wrong = 0.9498804807662964

